In [74]:
import pandas as pd
import re
pd.set_option('display.max_columns', None)

In [75]:
csv_file = 'dicom_files/dicomtags.csv'
unknown_view_file = 'dicom_files/unkwown_view.csv'
classify_view_file = 'dicom_files/classified_view.csv'
df = pd.read_csv(csv_file)

In [76]:
df['PixelData'].fillna(0, inplace=True)
df = df.fillna('')

In [77]:
df.head(3)

,Filename,AccessionNumber,AcquisitionDate,ImageType,InstitutionName,Manufacturer,ManufacturerModelName,ModalitiesInStudy,Modality,PatientID,ProtocolName,SeriesInstanceUID,SeriesTime,SOPClassUID,SOPInstanceUID,StationName,StudyID,StudyInstanceUID,StudyTime,StudyDescription,SeriesDescription,ViewPosition,PerformedProtocolCodeSequence,AcquisitionDeviceProcessingDescription,ViewCodeSequence,200B:1011,PixelData
0,../dicom_files/xray_chest_jpr/EX_out/0003IX174...,0003IX174800801L,,"['ORIGINAL', 'PRIMARY']",DASA - LAVOISIER INTERLAGOS I,KONICA MINOLTA,CS-7,DX,DX,7674221384,Torax PA/AP,1.2.392.200036.9107.307.32899.3289922112102036...,,1.2.840.10008.5.1.4.1.1.1.1,1.2.392.200036.9107.307.32899.20221121.163759....,CS-7,3289922112102036,1.2.840.55555.2050565141.20221121160016523.1326,163600,TORAX 2 INCIDENCIAS,PA,,"[(0008, 0100) Code Value ...",,"[(0008, 0100) Code Value ...",,9690840.0
1,../dicom_files/xray_chest_jpr/EX_out/0003IX174...,0003IX174800801L,,"['ORIGINAL', 'PRIMARY']",DASA - LAVOISIER INTERLAGOS I,KONICA MINOLTA,CS-7,DX,DX,7674221384,Torax LAT,1.2.392.200036.9107.307.32899.3289922112102036...,,1.2.840.10008.5.1.4.1.1.1.1,1.2.392.200036.9107.307.32899.20221121.162331....,CS-7,3289922112102036,1.2.840.55555.2050565141.20221121160016523.1326,163600,TORAX 2 INCIDENCIAS,LAT,,"[(0008, 0100) Code Value ...",,"[(0008, 0100) Code Value ...",,9690840.0
2,../dicom_files/xray_chest_jpr/EX_out/0003IQOWN...,0003IQOWNS00202B,,"['ORIGINAL', 'PRIMARY']",DASA DIAGNOSTICO DA AMERICA - ID# 2020225,KONICA MINOLTA,CS-7,DX,DX,7663570264,Torax PA/AP,1.2.392.200036.9107.307.23347.2334722072917151...,,1.2.840.10008.5.1.4.1.1.1.1,1.2.392.200036.9107.307.23347.20220729.105211....,CS-7,2334722072917151,1.2.840.55555.1020839050.20220729104324262.8663,105842,TORAX 1 INCIDENCIA,PA,,"[(0008, 0100) Code Value ...",,"[(0008, 0100) Code Value ...",,6931144.0


In [78]:
# drop undesired series
drop_series_descriptions = ['Carestream PACS Reports', 'KEY_IMAGES PR']

for drop_series in drop_series_descriptions:
    df = df[df['SeriesDescription'] != drop_series]

In [79]:
# drop images with less then 500 pixels
print(len(df))
df = df[df['PixelData'] > 500] 
print(len(df))

13256
12725


In [80]:
# concat tags containing important information
col_views = ['SeriesDescription', 'ViewPosition', 'AcquisitionDeviceProcessingDescription', 'ViewCodeSequence', '200B:1011']
df['concat_view'] = df['SeriesDescription'] + ' ' + df['ViewPosition'] + ' ' + df['AcquisitionDeviceProcessingDescription'] + ' ' + df['ViewCodeSequence']+ ' ' + df['200B:1011']
sorted(df['concat_view'].drop_duplicates().to_list())

['  ABDOME DD  ',
 '  ARCOS COSTAIS OBL D  ',
 '  ARCOS COSTAIS OBL E  ',
 '  ARCOS COSTAIS PA  ',
 '  ATM BOCA ABERTA  ',
 '  ATM BOCA FECHADA  ',
 '  COLUNA DORSAL F  ',
 '  COLUNA DORSAL P  ',
 '  COSTELAS FRN  ',
 '  LOMBO SACRA AP  ',
 '  LOMBO SACRA PERFIL  ',
 '  TORAX AP  ',
 '  TORAX OBL PA  ',
 '  TORAX OBLIQUAS PA  ',
 '  TORAX PA  ',
 '  TORAX PERFIL  ',
 " AP  [(0008, 0100) Code Value                          SH: 'R-10206'\n(0008, 0102) Coding Scheme Designator            SH: 'SRT'\n(0008, 0103) Coding Scheme Version               SH: ''\n(0008, 0104) Code Meaning                        LO: 'antero-posterior'] ",
 " LATERAL  [(0008, 0100) Code Value                          SH: 'R-102CD'\n(0008, 0102) Coding Scheme Designator            SH: 'SRT'\n(0008, 0103) Coding Scheme Version               SH: ''\n(0008, 0104) Code Meaning                        LO: 'lateral'] ",
 " OBLIQUE  [(0008, 0100) Code Value                          SH: 'R-102C1'\n(0008, 0102) Coding Scheme D

In [81]:
# drops images containing exclusion tags, e.g.: ATM, SEIOS DA FACE, WATER, etc.
drop_concat_views = ['ATM', r'OBL\w*', 'Apical', 'No Exam', 'SEIOS', 'LLO', 'RLO', r'WATER\w*', 'ABDOME', 'ARCOS COSTAIS',
              'COLUNA', r'COST\w*', r'LOMB\w*', 'SIG', 'Inspiracao', 'Expiracao']

for drop_view in drop_concat_views:
    pattern = re.compile(r'\b' + drop_view + r'\b', flags=re.IGNORECASE)
    df = df[~df['concat_view'].str.contains(pattern)]
print(len(df))
(df['concat_view'].drop_duplicates().to_list())

12466


["PA   [(0008, 0100) Code Value                          SH: 'R-10214'\n(0008, 0102) Coding Scheme Designator            SH: 'SRT'\n(0008, 0104) Code Meaning                        LO: 'postero-anterior'] ",
 "LAT   [(0008, 0100) Code Value                          SH: 'R-102CD'\n(0008, 0102) Coding Scheme Designator            SH: 'SRT'\n(0008, 0104) Code Meaning                        LO: 'lateral'] ",
 'PA    ',
 'PA PA   ',
 '  TORAX PA  ',
 "PA   [(0008, 0100) Code Value                          SH: 'R-102CD'\n(0008, 0102) Coding Scheme Designator            SH: 'SRT'\n(0008, 0104) Code Meaning                        LO: 'lateral'] ",
 "Chest PA postero-anterior^Factory1 [(0008, 0100) Code Value                          SH: 'R-10214'\n(0008, 0102) Coding Scheme Designator            SH: 'SNM3'\n(0008, 0104) Code Meaning                        LO: 'postero-anterior'] ",
 "AP   [(0008, 0100) Code Value                          SH: 'R-10206'\n(0008, 0102) Coding Scheme Designator    

In [82]:
def search_regex(pattern: str, text: str, flags=re.IGNORECASE) -> bool:
    # inserts word boundary
    pattern = re.compile(r'\b' + pattern.lower() + r'\b', flags=flags)
    pattern
    searched = re.search(pattern, text)
    return searched is not None

def classify_view(text: str)-> str:
    PA = ['postero-anterior', 'anterior-posterior', 'PA', 'AP']
    LAT = ['LATERAL', 'LAT', 'PERFIL', 'left-lateral', 'right-lateral']
    for search_word in LAT:
        if search_regex(search_word, text):
            return 'lat'

    for search_word in PA:
        if search_regex(search_word, text):
            return 'front'
        
        
    return ''

# classifies the view in front or lat
df['view'] = df['concat_view'].apply(classify_view)

In [83]:
df.view.value_counts()

front    9418
lat      3048
Name: view, dtype: int64

In [84]:
# shows images not classified
print(len(df[df['view'] == '']))
df[df['view'] == ''].head(5)

0


,Filename,AccessionNumber,AcquisitionDate,ImageType,InstitutionName,Manufacturer,ManufacturerModelName,ModalitiesInStudy,Modality,PatientID,ProtocolName,SeriesInstanceUID,SeriesTime,SOPClassUID,SOPInstanceUID,StationName,StudyID,StudyInstanceUID,StudyTime,StudyDescription,SeriesDescription,ViewPosition,PerformedProtocolCodeSequence,AcquisitionDeviceProcessingDescription,ViewCodeSequence,200B:1011,PixelData,concat_view,view


In [85]:
# saves unclassified images
df[df['view'] == ''].to_csv(unknown_view_file, index=False)
print(f'File saved {unknown_view_file}')

# saves classified images
df.to_csv(classify_view_file, index=False)
print(f'File saved {classify_view_file}')

File saved dicom_files/unkwown_view.csv
File saved dicom_files/classified_view.csv
